## DAY 3

## --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```


In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [6]:
import re

In [5]:
with open('../inputs/day-3.txt') as f:
    raw_input = f.read()
print(raw_input[:1000])
input_lines = raw_input.split('\n')
input_lines.pop()
input_lines[:10]

.....................................164.................429.35...........221....................................................34.........
........................464...........*.................................../.......53*.....954.763.....................114*.764..............
223............275.....................725.....$.........460....176............................*............+.................&.267.........
.........854..........919.798...............541.....302...................723......$...............196.......275......$....@....*...+2...388
..........@.......284*............429..211..........*..........633.............503..66......865.....*....234..........21....918.779..../....
...71....................40...856*........*.......................*.....438.......*.................636.#......671......................404.
.............585..........................866...971$.....869......548......#.456...907...146$.320%............+....*..354....*870...........
.............

['.....................................164.................429.35...........221....................................................34.........',
 '........................464...........*.................................../.......53*.....954.763.....................114*.764..............',
 '223............275.....................725.....$.........460....176............................*............+.................&.267.........',
 '.........854..........919.798...............541.....302...................723......$...............196.......275......$....@....*...+2...388',
 '..........@.......284*............429..211..........*..........633.............503..66......865.....*....234..........21....918.779..../....',
 '...71....................40...856*........*.......................*.....438.......*.................636.#......671......................404.',
 '.............585..........................866...971$.....869......548......#.456...907...146$.320%............+....*..354....*87

In [41]:
len(input_lines[100])

140

In [83]:
from collections import defaultdict 

In [84]:
schematic_dict = defaultdict(False)

for i, line in enumerate(input_lines):
    list_of_numbers = re.findall(r'\d+', line)
    reduced_line = line
    start_point_in_original_line = 0
    schematic_dict[i] = defaultdict(False)

    for number in list_of_numbers:
        occurrence_position_in_reduced_line = reduced_line.find(number)
        num_len = len(number)
        new_start_point = occurrence_position_in_reduced_line + num_len
        reduced_line = reduced_line[new_start_point:]
        occurrence_position = start_point_in_original_line + occurrence_position_in_reduced_line
        start_point_in_original_line = new_start_point + start_point_in_original_line
        
        schematic_dict[i][occurrence_position] = {'number' : int(number),
                                                  'len' : num_len,
                                                  'init' : occurrence_position,
                                                  'end' : occurrence_position + num_len - 1,
                                                  'list_of_positions' : list(range(occurrence_position, occurrence_position + num_len))
                                                  }
        
dict(list(schematic_dict.items())[:5])

TypeError: first argument must be callable or None

In [85]:
# Check if sorrounded by points

# Check left and right = same line and init-1 or end+1
# Check up and down = same init or end (and in number len) and line-1 or line+1
# Check all diagonals = init-1 or end+1 in lines line-1 or line+1
total_line_len = len(input_lines[0])
list_of_nums = []
for key, value in schematic_dict.items():
    for num_init in value.keys():
        num_end = value[num_init]['end']
        # Check left and right
        left_condition = (input_lines[key][num_init-1] == '.') if (num_init-1) >= 0 else True
        right_condition = (input_lines[key][num_end+1] == '.') if (num_end+1) < total_line_len else True
        # Check up and down and diagonals
        up_condition = all([(input_lines[key-1][pos]  == '.') if (pos>=0 and pos<len(input_lines[key-1])) else True for pos in range(num_init-1, num_end+2)]) if (key-1) >= 0 else True
        down_condition = all([input_lines[key+1][pos]  == '.' if (pos>=0 and pos<len(input_lines[key+1])) else True for pos in range(num_init-1, num_end+2)]) if (key+1) < len(input_lines) else True
        
        if not (left_condition and right_condition and up_condition and down_condition):
            list_of_nums.append(value[num_init]['number'])
        



In [86]:
answer = sum(list_of_nums)
answer

546563

## --- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```python 
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [67]:
possible_gear_dict = {}

for i, line in enumerate(input_lines):
    list_of_possible_gears = [number.start() for number in re.finditer(r'\*', line)]
    possible_gear_dict[i] = list_of_possible_gears
        
dict(list(possible_gear_dict.items())[:5])

{0: [], 1: [38, 84, 121], 2: [95], 3: [128], 4: [21, 52, 100]}

In [61]:
# Check if any digit is arround

total_line_len = len(input_lines[0])
gear = []
for key, value in gear_dict.items():
    for possible_gear_position in value:
        # Check left and right
        left_condition = (input_lines[key][possible_gear_position-1].isdigit()) if (possible_gear_position-1) >= 0 else True
        right_condition = (input_lines[key][possible_gear_position+1].isdigit()) if (possible_gear_position+1) < total_line_len else True
        # Check up and down and diagonals
        up_condition = all([(input_lines[key-1][pos]  == '.') if (pos>=0 and pos<len(input_lines[key-1])) else True for pos in range(num_init-1, num_end+2)]) if (key-1) >= 0 else True
        down_condition = all([input_lines[key+1][pos]  == '.' if (pos>=0 and pos<len(input_lines[key+1])) else True for pos in range(num_init-1, num_end+2)]) if (key+1) < len(input_lines) else True
        
        if not (left_condition and right_condition and up_condition and down_condition):
            list_of_nums.append(value[num_init]['number'])
        



['.....................................164.................429.35...........221....................................................34.........',
 '........................464...........*.................................../.......53*.....954.763.....................114*.764..............',
 '223............275.....................725.....$.........460....176............................*............+.................&.267.........',
 '.........854..........919.798...............541.....302...................723......$...............196.......275......$....@....*...+2...388',
 '..........@.......284*............429..211..........*..........633.............503..66......865.....*....234..........21....918.779..../....',
 '...71....................40...856*........*.......................*.....438.......*.................636.#......671......................404.',
 '.............585..........................866...971$.....869......548......#.456...907...146$.320%............+....*..354....*87

In [70]:
input_lines[0][38].isnumeric()

True

In [111]:
total_line_len = len(input_lines[0])
dict_of_possible_gear_positions = defaultdict(lambda: False)
for key, value in schematic_dict.items():
    for num_init in value.keys():
        num_end = value[num_init]['end']
        # Check left and right
        for pos in [num_init-1, num_end+1]:
            if pos >= 0 and pos < total_line_len:
                if input_lines[key][pos] == '*':
                    if dict_of_possible_gear_positions[(key, pos)]:
                        dict_of_possible_gear_positions[(key, pos)].append(value[num_init]['number'])
                    else:
                        dict_of_possible_gear_positions[(key, pos)] = [value[num_init]['number']]            
            
        # Check up and down and diagonals
        for pos in range(num_init-1, num_end+2):
            for line in [key-1, key+1]:
                if line >= 0 and line < len(input_lines):
                    if pos >= 0 and pos < len(input_lines[line]):
                        if input_lines[line][pos] == '*':
                            if dict_of_possible_gear_positions[(line, pos)]:
                                dict_of_possible_gear_positions[(line, pos)].append(value[num_init]['number'])
                            else:
                                dict_of_possible_gear_positions[(line, pos)] = [value[num_init]['number']]

In [112]:
dict(list(dict_of_possible_gear_positions.items())[:10])

{(1, 38): [164, 725],
 (1, 84): [53],
 (2, 95): [763],
 (1, 121): [114],
 (3, 128): [267, 779],
 (4, 21): [919, 284],
 (4, 52): [302],
 (4, 100): [196, 636],
 (5, 33): [429, 856],
 (5, 42): [211, 866]}

In [114]:
import math 
gear_ratios_list = [math.prod(value) for value in dict_of_possible_gear_positions.values() if len(value) > 1]
gear_ratios_list[:10]

[118900,
 207993,
 260996,
 124656,
 367224,
 182726,
 346884,
 456221,
 583099,
 280896]

In [115]:
answer = sum(gear_ratios_list)
answer

91031374